# Task 2 — Data Preparation & Integration (Outcome-Oriented)

**Input file:** `SA - Data for Task 2.xlsx`  
Sheets: **Work Order Data** + **Repair Data**  
Generated: **2025-12-16 06:33**

## What this notebook delivers (business outcomes)
- Identifies a reliable **Primary Key** for integration
- Cleans both datasets so integration is **safe and auditable**
- Produces a merged dataset that supports:
  - cost & labor analytics
  - failure vs fix correlation
  - trend analysis (Task 3)

> **Deliverable output:** `task2_merged_cleaned.xlsx` / `task2_merged_cleaned.csv`


In [ ]:
import pandas as pd, numpy as np, re
import matplotlib.pyplot as plt

path = r"/mnt/data/SA - Data for Task 2.xlsx"
wo_raw = pd.read_excel(path, sheet_name="Work Order Data")
rp_raw = pd.read_excel(path, sheet_name="Repair Data")
print("Work Order shape:", wo_raw.shape)
print("Repair shape:", rp_raw.shape)
wo_raw.head()

---
## 1) Primary Key identification
**Why:** A wrong key creates wrong joins → wrong insights → wrong decisions.

We evaluate candidates:
- `Primary Key`
- `Order No`
- `Segment Number`
- `Order No + Segment Number`

**Outcome:** We choose the key with **lowest nulls**, **lowest duplicates**, and real-world interpretability.


In [ ]:
pk_stats = []
for name, df in [("WorkOrder", wo_raw), ("Repair", rp_raw)]:
    for key in ["Primary Key", "Order No", "Segment Number", "Order No + Segment Number"]:
        if key == "Order No + Segment Number":
            combo = df["Order No"].astype(str) + "__" + df["Segment Number"].astype(str)
            pk_stats.append({
                "dataset": name,
                "key_candidate": key,
                "rows": len(df),
                "unique": combo.nunique(dropna=True),
                "duplicates": int(combo.duplicated().sum()),
                "nulls": int((df["Order No"].isna() | df["Segment Number"].isna()).sum())
            })
        else:
            pk_stats.append({
                "dataset": name,
                "key_candidate": key,
                "rows": len(df),
                "unique": df[key].nunique(dropna=True),
                "duplicates": int(df[key].duplicated().sum()),
                "nulls": int(df[key].isna().sum())
            })
pk_stats_df = pd.DataFrame(pk_stats)
pk_stats_df

### Decision
If `Primary Key` is unique and non-null across both sheets, it is the best choice because:
- it is designed for integration
- it avoids edge cases where Order numbers repeat

If not, we use the composite key `Order No + Segment Number`.


In [ ]:
# Simple automatic decision
use_primary_key = (
    pk_stats_df.query("key_candidate=='Primary Key' and duplicates==0 and nulls==0").shape[0] == 2
)
chosen_pk = "Primary Key" if use_primary_key else "Order No + Segment Number"
print("Chosen key:", chosen_pk)

---
## 2) Data cleaning (both datasets)
**Why:** Integration only works if both sides have consistent formats.

Cleaning performed:
- text normalization (trim, whitespace)
- date parsing (`Order Date`, `Invoice Date`)
- numeric coercion (`Qty`, `Revenue`, `Cost`, `Actual Hours`, etc.)
- missing handling (categorical→`UNKNOWN`)
- deduplication on chosen PK

**Outcome:** Datasets become join-safe + analytics-safe.


In [ ]:
def normalize_text(x):
    if pd.isna(x): return ""
    x = str(x).replace("\n"," ").replace("\r"," ")
    return re.sub(r"\s+"," ", x).strip()

def coerce_numeric(series):
    s = series.astype(str).str.replace(",", "", regex=False)
    s = s.str.replace("$","", regex=False).str.strip()
    return pd.to_numeric(s, errors="coerce")

wo = wo_raw.copy()
rp = rp_raw.copy()

for df in (wo, rp):
    for c in df.select_dtypes(include=["object"]).columns:
        df[c] = df[c].apply(normalize_text).replace("", np.nan)

wo["Order Date"] = pd.to_datetime(wo["Order Date"], errors="coerce")
rp["Invoice Date"] = pd.to_datetime(rp["Invoice Date"], errors="coerce")

for col in ["Model Year", "Meter 1 Reading"]:
    if col in wo.columns:
        wo[col] = coerce_numeric(wo[col])

for col in ["Qty","Revenue","Cost","Actual Hours","Segment Total $"]:
    if col in rp.columns:
        rp[col] = coerce_numeric(rp[col])

# fill categoricals
for df in (wo, rp):
    for c in df.columns:
        if df[c].dtype == "object":
            df[c] = df[c].fillna("UNKNOWN")

# dedupe
key_cols = ["Primary Key"] if chosen_pk=="Primary Key" else ["Order No","Segment Number"]
wo_before, rp_before = len(wo), len(rp)
wo = wo.drop_duplicates(subset=key_cols, keep="first")
rp = rp.drop_duplicates(subset=key_cols, keep="first")
print("WO duplicates removed:", wo_before - len(wo))
print("RP duplicates removed:", rp_before - len(rp))

---
## 3) Integration (Merge)
**Join type selected:** **Left join** from Work Orders → Repair Data.

**Why left join?**
- Work order is the “master” business event (complaint/cause/correction)
- Repair rows may be missing for some work orders (e.g., incomplete billing)
- Left join preserves the full work order universe for analysis

**Outcome:** A comprehensive dataset with both failure context and financial/labor details.


In [ ]:
if chosen_pk=="Primary Key":
    merged = wo.merge(rp, on="Primary Key", how="left", suffixes=("_WO","_RP"))
else:
    merged = wo.merge(rp, on=["Order No","Segment Number"], how="left", suffixes=("_WO","_RP"))

print("Merged shape:", merged.shape)
merged.head()

### Implications of other join types
- **Inner join:** drops work orders with no matching repair records (risk: biased analysis)
- **Right join:** not suitable because repair records without work order context are hard to interpret
- **Full outer join:** keeps everything but requires more reconciliation (good for audits, not first-pass analytics)


In [ ]:
# Save merged outputs
merged.to_excel(r"/mnt/data/axionray_task2_3/task2_merged_cleaned.xlsx", index=False)
merged.to_csv(r"/mnt/data/axionray_task2_3/task2_merged_cleaned.csv", index=False)
print("Saved:", r"/mnt/data/axionray_task2_3/task2_merged_cleaned.xlsx")
print("Saved:", r"/mnt/data/axionray_task2_3/task2_merged_cleaned.csv")